In [ ]:
import sys
sys.path.append('..')

In [ ]:
import os
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

In [ ]:
from DomainPrediction.utils import helper
from DomainPrediction.eval import metrics
from DomainPrediction.esm.esm2 import ESM2

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from scipy import stats
from sklearn.model_selection import train_test_split

In [ ]:
import warnings

warnings.filterwarnings("ignore", ".*Consider increasing the value of the `num_workers` argument*")
warnings.filterwarnings("ignore", ".*Set a lower value for log_every_n_steps*")

#### Define Functions

In [ ]:
esm2 = ESM2(model_path='/data/users/kgeorge/workspace/esm2/checkpoints/esm2_t33_650M_UR50D.pt', device='gpu')

In [ ]:
def get_embeddings_mean(sequences):
    embeddings = []
    for seq in tqdm(sequences):
        rep = esm2.get_res(sequence=seq)
        embeddings.append(rep['representations'][33][:,1:-1,:].mean(1).cpu().numpy())

    embeddings = np.concatenate(embeddings, axis=0)

    return embeddings

def get_embeddings_mean_batch(sequences, batch_size=3):
    embeddings = []
    for i in tqdm(range(0, len(sequences), batch_size)):
        seqs = sequences[i:i+batch_size]
        rep, batch_lens = esm2.get_res_batch(sequences=seqs)
        assert (batch_lens == batch_lens[0]).all() == True
        embeddings.append(rep['representations'][33][:,1:-1,:].mean(1).cpu().numpy())

    embeddings = np.concatenate(embeddings, axis=0)

    return embeddings

def get_embeddings_full(sequences):
    embeddings = []
    for seq in tqdm(sequences):
        rep = esm2.get_res(sequence=seq)
        embeddings.append(rep['representations'][33][:,1:-1,:].cpu().numpy()[0])

    return embeddings

def one_hot_encode(sequences: list[str]) -> np.ndarray:
    """Encode a protein sequence as a one-hot array."""
    embeddings = []
    for seq in tqdm(sequences):
        amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
        aa_to_index = {aa: i for i, aa in enumerate(amino_acids)}
        one_hot = np.zeros((len(seq), len(amino_acids)))
        for i, aa in enumerate(seq):
            if aa in amino_acids:
                one_hot[i, aa_to_index[aa]] = 1
    
        embeddings.append(one_hot.flatten())  

    embeddings = np.stack(embeddings, axis=0)

    return embeddings

In [ ]:
class RFSurrogate():
    def __init__(self) -> None:

        self.model = RandomForestRegressor(n_estimators=100, criterion='friedman_mse', max_depth=None, min_samples_split=2,
                                            min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=1.0,
                                            max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False,
                                            n_jobs=None, random_state=1, verbose=0, warm_start=False, ccp_alpha=0.0,
                                            max_samples=None)
    
    def train(self, X, y, val=None, debug=True):
        '''
            X - embeddings from esm2
            X - shape (n, features)
            y - shape (n, )
        '''
        # if val:
        #     idx = np.arange(X.shape[0])
        #     train_idx, val_idx = train_test_split(idx, test_size=0.2)
        #     _ = self.model.fit(X[train_idx], y[train_idx])

        #     if debug:
        #         self.print_eval(X[train_idx], y[train_idx], label='train')
        #         self.print_eval(X[val_idx], y[val_idx], label='val')
        # else:
        _ = self.model.fit(X, y)
        if debug:
            self.print_eval(X, y, label='train')
            if val is not None:
                X_val, y_val = val
                self.print_eval(X_val, y_val, label='val')

    
    def print_eval(self, X, y, label='set'):
        ypred = self.model.predict(X)
        mse = mean_squared_error(ypred, y)
        corr = stats.spearmanr(ypred, y)

        print(f'{label}: mse = {mse}, spearman correlation = {corr.statistic}')

    def predict(self, X):

        return self.model.predict(X)

In [ ]:
class RidgeSurrogate():
    def __init__(self) -> None:

        self.model = Ridge(alpha=1.0, fit_intercept=True, random_state=1)
    
    def train(self, X, y, val=None, debug=True):
        '''
            X - embeddings from esm2
            X - shape (n, features)
            y - shape (n, )
        '''
        # if val:
        #     idx = np.arange(X.shape[0])
        #     train_idx, val_idx = train_test_split(idx, test_size=0.2)
        #     _ = self.model.fit(X[train_idx], y[train_idx])

        #     if debug:
        #         self.print_eval(X[train_idx], y[train_idx], label='train')
        #         self.print_eval(X[val_idx], y[val_idx], label='val')
        # else:
        #     _ = self.model.fit(X, y)
        #     if debug:
        #         self.print_eval(X, y, label='train')

        _ = self.model.fit(X, y)
        if debug:
            self.print_eval(X, y, label='train')
            if val is not None:
                X_val, y_val = val
                self.print_eval(X_val, y_val, label='val')
    
    def print_eval(self, X, y, label='set'):
        ypred = self.model.predict(X)
        mse = mean_squared_error(ypred, y)
        corr = stats.spearmanr(ypred, y)

        print(f'{label}: mse = {mse}, spearman correlation = {corr.statistic}')

    def predict(self, X):

        return self.model.predict(X)

In [ ]:
from sklearn.neural_network import MLPRegressor
class EMLPSurrogate():
    def __init__(self) -> None:

        self.model = MLPRegressor(hidden_layer_sizes=(5), max_iter=1000, activation='relu', solver='adam', alpha=0.001,
                             batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5,
                             momentum=0.9, nesterovs_momentum=True, shuffle=True, random_state=1, tol=0.0001,
                             verbose=False, warm_start=False, early_stopping=False, validation_fraction=0.1, beta_1=0.9,
                             beta_2=0.999, epsilon=1e-08)
    
    def train(self, X, y, val=True, debug=True):
        '''
            X - embeddings from esm2
            X - shape (n, features)
            y - shape (n, )
        '''
        if val:
            idx = np.arange(X.shape[0])
            train_idx, val_idx = train_test_split(idx, test_size=0.2)
            _ = self.model.fit(X[train_idx], y[train_idx])

            if debug:
                self.print_eval(X[train_idx], y[train_idx], label='train')
                self.print_eval(X[val_idx], y[val_idx], label='val')
        else:
            _ = self.model.fit(X, y)
            if debug:
                self.print_eval(X, y, label='train')
    
    def print_eval(self, X, y, label='set'):
        ypred = self.model.predict(X)
        mse = mean_squared_error(ypred, y)
        corr = stats.spearmanr(ypred, y)

        print(f'{label}: mse = {mse}, spearman correlation = {corr.statistic}')

    def predict(self, X):

        return self.model.predict(X)

In [ ]:
class ProteinFunDataset(Dataset):
    def __init__(self, X, y):
        self.X, self.y = X, y
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class MLPSurrogate(pl.LightningModule):
    def __init__(self, inp_size=1280, hidden_size=512, 
                 config={'epoch': 10, 
                         'batch_size': 16}
                ) -> None:
        super().__init__()
        self.config = config
        # self.mlp = nn.Sequential(
        #     nn.Linear(inp_size, hidden_size),
        #     nn.ReLU(),
        #     nn.Linear(hidden_size, 1)
        # )
        self.mlp = nn.Sequential(
            nn.LayerNorm(1280),
            nn.Linear(1280, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1280),
            nn.ReLU(),
            nn.Linear(1280, 1)
        )
        self.accumulate_batch_loss_train = []
        self.accumulate_batch_loss_val = []
        self.debug=True

    def forward(self, x):
        x = self.mlp(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        # y_hat = self.mlp(x)
        # print(y_hat)
        # print(y_hat.flatten())
        # print(y)
        # loss = nn.functional.mse_loss(y_hat.flatten(), y)
        loss = nn.functional.mse_loss(y_hat.flatten(), y)
        self.log("train/loss", loss, on_step=True, on_epoch=True)
        self.accumulate_batch_loss_train.append(loss.item())
        # print(loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        # y_hat = self.mlp(x)
        loss = nn.functional.mse_loss(y_hat.flatten(), y)
        self.log("val/loss", loss, on_step=True, on_epoch=True)
        self.accumulate_batch_loss_val.append(loss.item())
    
    @staticmethod
    def trainmodel(model, X, y, val=True, debug=True):
        '''
            X - embeddings from esm2
            X - shape (n, features)
            y - shape (n, )
        '''
        model.debug = debug
        if val:
            idx = np.arange(X.shape[0])
            train_idx, val_idx = train_test_split(idx, test_size=0.2)
            train_dataset = ProteinFunDataset(X[train_idx], y[train_idx])
            val_dataset = ProteinFunDataset(X[val_idx], y[val_idx])
            train_loader = DataLoader(train_dataset, batch_size=model.config['batch_size'], shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=model.config['batch_size'], shuffle=False)

            earlystopping_callback = EarlyStopping(monitor="val/loss", patience=10, verbose=False, mode="min")

            trainer = pl.Trainer(max_epochs=model.config['epoch'], callbacks=[earlystopping_callback],
                                 accelerator="auto",
                                 enable_progress_bar=False,
                                 enable_model_summary=False
                                 )
            trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

            ## Needs to change - we need to load the least val loss model
            y_pred = model.predict(X[val_idx])
            val_mse = mean_squared_error(y_pred, y[val_idx])
            print(f'Train end val mse: {val_mse}')

        else:
            raise Exception("Needs Fix")
            train_dataset = ProteinFunDataset(X, y)
            train_loader = DataLoader(train_dataset)

            trainer = pl.Trainer(max_epochs=95, 
                                 enable_progress_bar=False,
                                 accelerator="auto"
                                 )
            trainer.fit(model=model, train_dataloaders=train_loader)

    def on_train_epoch_start(self):
        self.accumulate_batch_loss_train.clear()
        self.accumulate_batch_loss_val.clear()
    
    def on_train_epoch_end(self):
        if self.current_epoch % self.config['print_every_n_epoch'] == 0 and self.debug:
            print(f'Epoch: {self.current_epoch}: train mse: {np.mean(self.accumulate_batch_loss_train)} val mse: {np.mean(self.accumulate_batch_loss_val)}')

    def on_train_end(self):
        print(f'Epoch: {self.current_epoch}: train mse: {np.mean(self.accumulate_batch_loss_train)} val mse: {np.mean(self.accumulate_batch_loss_val)}')

    def predict(self, X):
        '''
            X is numpy array
        '''
        with torch.no_grad():
            y = self(torch.tensor(X))
        return y.numpy().flatten()
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

#### Load Data

In [ ]:
root = '../..'
data_path = os.path.join(root, 'Data/al_test_experiments/Evolvepro')

In [ ]:
file_name = os.path.join(data_path, 'brenan.csv')
df = pd.read_csv(file_name)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
len(df['seq'][0])

#### Tests

In [ ]:
## ESM2 mean-pooled embeddings

# embeddings = get_embeddings_mean(df['seq'])
# file_name = os.path.join(data_path, 'brenan_embeddings.npy')
# np.save(file_name, embeddings)

file_name = os.path.join(data_path, 'brenan_embeddings.npy')
embeddings = np.load(file_name)

In [ ]:
# embeddings = one_hot_encode(df['seq'])

In [ ]:
embeddings.shape

In [ ]:
np.random.seed(0)

In [ ]:
test_split = 0.2
num_seq_in_pos = 19
block_size = 10
num_blocks = int(df.shape[0]*test_split // (num_seq_in_pos * block_size) + 1)
positions = df['pos'].unique()
step_size = len(positions) // num_blocks
block_indices = [i for i in range(0, len(positions) - block_size + 1, step_size)][:num_blocks]
blocks = [positions[i:i + block_size] for i in block_indices]

for _block in blocks:
    assert len(_block) == 10
    for i in _block:
        assert i in positions

test_positions = np.concatenate(blocks)
test_indices = np.array(df[df['pos'].isin(test_positions)].index)

val_split = 0.1
n_pos_val = int((~df['pos'].isin(test_positions)).sum()*val_split // num_seq_in_pos + 1)
val_positions = np.random.choice(df.loc[~df['pos'].isin(test_positions), 'pos'].unique(), n_pos_val, replace=False)
for i in val_positions:
    assert i not in test_positions
val_indices = np.array(df[df['pos'].isin(val_positions)].index)
train_indices = np.array(df[~df['pos'].isin(np.concatenate([val_positions, test_positions]))].index)

In [ ]:
print(f'Total size: {df.shape[0]}')
print(f'train size: {train_indices.shape[0]} ({round(train_indices.shape[0]*100/df.shape[0], 2)}%)')
print(f'val size  : {val_indices.shape[0]} ({round(val_indices.shape[0]*100/df.shape[0], 2)}%)')
print(f'test size : {test_indices.shape[0]} ({round(test_indices.shape[0]*100/df.shape[0], 2)}%)')

In [ ]:
scaled = False
if scaled:
    property_label = 'function_scaled'
else:
    property_label = 'function'

In [ ]:
X = embeddings
y = df[property_label].to_numpy().astype(np.float32)

In [ ]:
X_train = X[train_indices]
y_train = y[train_indices]

X_val = X[val_indices]
y_val = y[val_indices]

X_test = X[test_indices]
y_test = y[test_indices]

In [ ]:
surrogate = RidgeSurrogate()
surrogate.train(X=X_train, y=y_train, val=(X_val, y_val))

In [ ]:
y_train_pred = surrogate.predict(X_train)
y_val_pred = surrogate.predict(X_val)
y_test_pred = surrogate.predict(X_test)
fig, ax = plt.subplots(1,3, figsize=(10,3), layout='constrained')
ax[0].plot(y_train, y_train_pred, '.', alpha=0.5)
ax[1].plot(y_val, y_val_pred, '.', alpha=0.5)
ax[2].plot(y_test, y_test_pred, '.', alpha=0.5)

mse = mean_squared_error(y_train, y_train_pred)
corr = stats.spearmanr(y_train, y_train_pred)
s_corr = round(corr.statistic, 2)
ax[0].set_title(f'Train \nmse : {str(round(mse, 2))} \nspearman correlation = {s_corr}')

mse = mean_squared_error(y_val, y_val_pred)
corr = stats.spearmanr(y_val, y_val_pred)
s_corr = round(corr.statistic, 2)
ax[1].set_title(f'Val \nmse : {str(round(mse, 2))} \nspearman correlation = {s_corr}')

mse = mean_squared_error(y_test, y_test_pred)
corr = stats.spearmanr(y_test, y_test_pred)
s_corr = round(corr.statistic, 2)
ax[2].set_title(f'Test \nmse : {str(round(mse, 2))} \nspearman correlation = {s_corr}')

for i in range(3):
    ax[i].set_xlabel('True')
    ax[i].set_ylabel('Pred')

plt.show()

In [ ]:
surrogate = RFSurrogate()
surrogate.train(X=X_train, y=y_train, val=(X_val, y_val))

In [ ]:
y_train_pred = surrogate.predict(X_train)
y_val_pred = surrogate.predict(X_val)
y_test_pred = surrogate.predict(X_test)
fig, ax = plt.subplots(1,3, figsize=(10,3), layout='constrained')
ax[0].plot(y_train, y_train_pred, '.', alpha=0.5)
ax[1].plot(y_val, y_val_pred, '.', alpha=0.5)
ax[2].plot(y_test, y_test_pred, '.', alpha=0.5)

mse = mean_squared_error(y_train, y_train_pred)
corr = stats.spearmanr(y_train, y_train_pred)
s_corr = round(corr.statistic, 2)
ax[0].set_title(f'Train \nmse : {str(round(mse, 2))} \nspearman correlation = {s_corr}')

mse = mean_squared_error(y_val, y_val_pred)
corr = stats.spearmanr(y_val, y_val_pred)
s_corr = round(corr.statistic, 2)
ax[1].set_title(f'Val \nmse : {str(round(mse, 2))} \nspearman correlation = {s_corr}')

mse = mean_squared_error(y_test, y_test_pred)
corr = stats.spearmanr(y_test, y_test_pred)
s_corr = round(corr.statistic, 2)
ax[2].set_title(f'Test \nmse : {str(round(mse, 2))} \nspearman correlation = {s_corr}')

for i in range(3):
    ax[i].set_xlabel('True')
    ax[i].set_ylabel('Pred')

plt.show()

In [ ]:
# surrogate = EMLPSurrogate()
# surrogate.train(X=X, y=y)

In [ ]:
# ypred = surrogate.predict(X)
# plt.plot(y, ypred, '.')
# mse = mean_squared_error(y, ypred)
# corr = stats.spearmanr(y, ypred)
# s_corr = round(corr.statistic, 2)
# plt.title(f'mse : {str(round(mse, 2))} \nspearman correlation = {s_corr}')
# if scaled:
#     _min = 0
#     _max = max(y) + 0.1
# else:
#     _min = min(y) - 0.1
#     _max = max(y) + 0.1

# plt.xlim(_min, _max)
# plt.ylim(_min, _max)
# plt.show()

In [ ]:
config = {'epoch': 500, 'batch_size': 32, 'print_every_n_epoch': 10}
surrogate = MLPSurrogate(config=config)
MLPSurrogate.trainmodel(model=surrogate, X=X.astype(np.float32), y=y)

In [ ]:
surrogate

In [ ]:
ypred = surrogate.predict(X.astype(np.float32))
fig, ax = plt.subplots(1,2, figsize=(7,3), layout='constrained')
ax[0].plot(y, ypred, '.')
ax[1].plot(y, ypred, '.')
mse = mean_squared_error(y, ypred)
corr = stats.spearmanr(y, ypred)
s_corr = round(corr.statistic, 2)
ax[0].set_title(f'yaxis \nmse : {str(round(mse, 2))} \nspearman correlation = {s_corr}')
ax[1].set_title(f'yaxis-min-max \nmse : {str(round(mse, 2))} \nspearman correlation = {s_corr}')
if scaled:
    _min = 0
    _max = max(y) + 0.1
else:
    _min = min(y) - 0.1
    _max = max(y) + 0.1

ax[1].set_xlim(_min, _max)
ax[1].set_ylim(_min, _max)
plt.show()